In [ ]:
# !unzip data.zip

In [ ]:
# !pip3 install --upgrade pandas

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Входные данные

In [ ]:
import pandas as pd
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector, ColumnTransformer

PATH_TO_DATA = 'data'

In [ ]:
df = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_data.train'))
y = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_labels.train'), header=None, squeeze=True)

In [ ]:
df.shape, y.shape

((40000, 230), (40000,))

In [ ]:
class NanColumnsDropper(BaseEstimator, TransformerMixin):
    def __init(self):
        super().__init__()

    def fit(self, X):
        self.nan_cols = X.loc[:, X.isna().all()].columns
        return self

    def transform(self, X):
        X_transformed = X.drop(self.nan_cols, axis=1)
        self.cols = X_transformed.columns
        return X_transformed

In [ ]:
import sklearn

sklearn.__version__

'0.22.2.post1'

In [ ]:
numeric_transformer = SimpleImputer(strategy='most_frequent')

categorical_transformer = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal_encoder', OrdinalEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude="object")),
    ('cat', categorical_transformer, selector(dtype_include="object"))
])

preprocessor_pipeline = Pipeline([
    ('nan_columns_dropper', NanColumnsDropper()),
    ('preprocessor', preprocessor)
])

In [ ]:
# Отделим hold-out dataset
df_train, df_ho, y_train, y_valid = train_test_split(df, y, test_size=5000)
# df_train_ho.to_csv(os.path.join(PATH_TO_DATA, 'hold_out_train_dataset.csv'))

In [ ]:
%%time
X_train = preprocessor_pipeline.fit_transform(df_train)

CPU times: user 6.08 s, sys: 227 ms, total: 6.3 s
Wall time: 6.29 s


In [ ]:
X_train = pd.DataFrame(X_train, columns=preprocessor_pipeline['nan_columns_dropper'].cols).astype('int')
X_train

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,Var12,Var13,Var14,Var16,Var17,Var18,Var19,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var33,Var34,Var35,Var36,Var37,Var38,Var40,Var41,Var43,Var44,Var45,Var46,Var47,...,Var189,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0,0,0,0,0,644,7,0,0,8,0,8,0,0,0,0,0,236,295,0,0,32,0,0,166,0,5,0,0,5,0,0,368286,0,0,0,0,2985,0,0,...,276,0,0,203,24,1,21,0,199,1511,1441,6047,1,3276,0,87,0,17,10,0,5,0,32,0,10179,0,1231,6449,1,11,2339,4,3064,0,0,0,18,2,7,0
1,0,0,0,0,0,2737,7,0,0,8,0,328,0,0,0,0,0,360,450,0,4,192,0,0,253,0,5,0,0,0,0,0,4468200,0,0,0,0,2985,0,0,...,252,0,0,154,0,1,21,0,2,869,335,3574,1,156,2,79,0,3,10,1,5,0,43,0,11084,0,668,4289,1,11,1451,4,2937,0,0,0,14,2,2,0
2,0,0,0,0,0,1057,7,0,0,8,0,320,0,0,0,0,0,20,25,0,0,8,0,0,186,0,5,0,0,0,0,0,0,0,0,0,0,2985,0,0,...,282,0,0,286,24,1,21,0,1,595,828,7889,1,1455,0,34,1,10,10,0,5,0,43,0,1963,0,1512,9738,1,14,2789,4,3158,3,0,0,9,2,7,1
3,0,0,0,0,0,1330,7,0,0,8,0,1876,0,0,0,0,0,184,230,0,6,144,0,0,133,0,5,0,0,0,0,0,82716,0,0,0,0,2985,0,0,...,276,0,0,328,0,1,21,0,41,1519,3623,6791,1,4193,0,47,1,14,9,0,5,0,26,0,1299,0,1479,4075,1,11,1799,6,554,0,0,0,13,1,18,0
4,0,0,0,0,0,1316,7,0,0,8,0,2096,0,0,0,0,0,184,230,0,6,120,0,0,133,0,5,0,0,0,0,0,3732,0,0,0,0,2985,0,0,...,234,0,0,91,24,1,21,0,36,2007,1001,10452,1,2143,0,20,1,11,10,0,5,0,43,0,3199,0,668,7244,1,11,2997,4,766,0,0,0,13,2,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,0,0,0,0,0,0,7,0,0,8,0,0,0,0,30,108,0,0,0,0,0,0,0,0,166,0,5,0,0,0,0,661041,0,0,0,0,0,2985,0,0,...,276,0,0,343,24,1,21,0,70,3014,1630,11686,1,1360,0,70,1,6,5,0,5,0,17,0,1196,0,389,1999,1,11,3371,3,2094,0,0,0,5,0,7,0
34996,0,0,0,0,0,2429,7,0,0,8,0,68,0,0,0,0,0,852,1065,0,26,536,0,0,320,0,5,0,0,0,0,0,6329700,0,0,0,0,2985,0,0,...,294,0,0,139,24,1,21,0,25,76,942,1874,1,4615,0,35,0,17,10,0,5,0,32,0,10353,0,390,896,0,11,2192,4,126,0,0,1,1,2,7,0
34997,0,0,0,0,0,84,0,0,0,8,0,0,0,0,0,0,0,104,130,0,0,40,0,0,166,0,5,0,0,0,0,0,1025586,0,0,0,27,2985,0,0,...,276,0,0,78,24,1,21,0,2,2707,828,11686,1,234,2,58,1,6,10,1,5,1,32,0,1196,0,1324,1002,1,11,1519,4,879,0,0,0,10,2,7,0
34998,0,0,0,0,0,0,7,0,0,8,0,0,0,0,5,0,0,0,0,0,0,0,0,0,166,0,5,0,0,0,0,0,0,0,0,0,0,2985,0,0,...,276,0,0,263,24,1,21,0,17,3096,2828,11686,1,3199,0,88,0,6,10,0,5,1,32,0,1196,0,1215,8125,1,11,2971,4,759,0,0,0,10,2,7,0


In [ ]:
%%time
X_valid = preprocessor_pipeline.fit_transform(df_ho)

CPU times: user 375 ms, sys: 1.79 ms, total: 377 ms
Wall time: 380 ms


In [ ]:
X_valid = pd.DataFrame(X_valid, columns=preprocessor_pipeline['nan_columns_dropper'].cols).astype('int')
X_valid

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,Var12,Var13,Var14,Var16,Var17,Var18,Var19,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var33,Var34,Var35,Var36,Var37,Var38,Var40,Var41,Var43,Var44,Var45,Var46,Var47,...,Var189,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0,0,0,0,0,1309,7,0,0,8,0,4,0,0,0,0,0,520,650,0,0,192,0,0,200,0,5,0,0,0,0,0,2862894,0,0,0,0,0,0,0,...,420,9,0,126,0,1,15,0,80,982,602,1888,1,547,0,28,0,16,9,0,5,0,24,0,36,0,284,776,0,7,119,4,973,3,0,1,22,2,2,1
1,0,0,0,0,0,175,7,0,0,8,0,4,0,0,0,0,0,156,195,0,2,176,0,0,146,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,...,282,9,0,32,19,1,15,0,57,1082,1064,1739,1,2533,0,54,2,0,9,0,5,0,24,0,2306,0,51,292,0,7,1428,4,619,0,0,1,19,2,8,0
2,0,0,0,0,0,1141,7,0,0,8,0,5276,0,0,0,0,0,324,405,0,8,80,0,0,210,0,5,0,0,0,0,0,3676842,0,0,0,0,0,0,0,...,282,9,0,224,0,1,3,0,31,1053,688,2107,1,173,0,94,1,18,3,0,5,0,36,0,2027,0,10,1584,0,7,122,4,1006,0,0,1,20,3,15,0
3,0,0,0,0,0,798,7,0,0,8,0,356,0,0,0,0,0,104,130,0,2,120,0,0,166,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,...,282,9,0,135,19,1,15,0,57,836,305,1657,1,44,0,43,2,5,9,0,5,0,24,0,1170,0,402,1046,0,7,1220,4,552,0,0,1,7,2,8,0
4,0,0,0,0,0,0,7,0,0,8,0,0,0,0,0,0,0,128,0,0,0,0,0,0,166,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,...,282,9,0,217,19,1,15,2,2,468,314,1173,1,106,2,52,0,5,9,1,5,1,24,0,1003,0,64,1725,0,7,855,4,1267,0,0,0,2,2,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,0,0,0,0,336,0,0,0,8,0,44,0,0,0,0,0,0,0,0,0,0,0,0,133,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,...,282,9,0,178,19,1,15,0,89,1445,675,1411,1,1010,0,12,0,13,9,0,5,0,33,0,1046,0,1,1777,1,7,865,4,144,0,0,2,12,2,8,0
4996,0,0,0,0,0,0,7,0,0,8,0,0,0,121,0,0,0,128,0,0,0,0,0,0,166,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,...,282,9,0,120,19,1,15,0,16,704,1031,1173,1,1151,0,43,0,5,3,0,5,1,24,0,1003,0,490,3010,0,7,260,4,1350,0,0,0,7,3,8,0
4997,0,0,0,0,0,2002,7,0,0,8,0,2348,0,0,0,0,0,392,490,0,0,232,0,0,200,0,5,0,0,0,0,0,1441302,0,0,0,0,0,0,0,...,282,9,0,165,6,1,3,0,75,1053,706,1350,1,1974,0,92,1,18,3,0,5,0,36,0,1729,0,382,477,0,7,122,4,1006,3,0,1,4,3,10,1
4998,0,0,0,0,0,245,7,0,0,8,0,148,0,0,0,0,0,100,125,0,0,80,0,0,226,0,5,0,0,0,0,0,53574,0,0,0,0,0,0,0,...,234,9,0,249,19,1,15,0,30,390,672,168,1,39,0,60,1,9,9,0,5,0,33,0,1626,0,1,2687,1,7,241,4,890,0,0,0,3,2,8,0


# Модель

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, roc_auc_score

In [ ]:
cv = StratifiedShuffleSplit(n_splits=8, random_state=2179)

## LogisticRegression

In [ ]:
lr_cv = cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')
lr_cv, lr_cv.mean()

(array([0.49043329, 0.51152184, 0.49272436, 0.51360161, 0.48530627,
        0.51666667, 0.49531339, 0.47154558]), 0.497139126305793)

In [ ]:
lr_cv = cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')
lr_cv, lr_cv.mean()

(array([0.00746269, 0.02264151, 0.        , 0.01428571, 0.        ,
        0.02230483, 0.        , 0.        ]), 0.008336842875074423)

In [ ]:
lr_cv = cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')
lr_cv, lr_cv.mean()

(array([0.125     , 0.6       , 0.        , 0.1       , 0.        ,
        0.33333333, 0.        , 0.        ]), 0.14479166666666665)

In [ ]:
lr_cv = cross_val_score(estimator=LogisticRegression(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')
lr_cv, lr_cv.mean()

(array([0.00384615, 0.01153846, 0.        , 0.00769231, 0.        ,
        0.01153846, 0.        , 0.        ]), 0.004326923076923077)

In [ ]:
logistic_regression = LogisticRegression(random_state=2179)
logistic_regression.fit(X_train, y_train)
print(classification_report(y_valid, logistic_regression.predict(X_valid)))

              precision    recall  f1-score   support

          -1       0.92      0.99      0.96      4620
           1       0.07      0.01      0.01       380

    accuracy                           0.92      5000
   macro avg       0.50      0.50      0.49      5000
weighted avg       0.86      0.92      0.88      5000



In [ ]:
roc_auc_score(y_valid, logistic_regression.predict_proba(X_valid)[:, 1])

0.5072818409660514

## RidgeClassifier

In [ ]:
scores = cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')
scores, scores.mean()

(array([0.52948362, 0.49871557, 0.51535494, 0.5028312 , 0.49508547,
        0.5234283 , 0.50317901, 0.50207858]), 0.508769586894587)

In [ ]:
scores = cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')
scores, scores.mean()

(array([0., 0., 0., 0., 0., 0., 0., 0.]), 0.0)

In [ ]:
scores = cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')
scores, scores.mean()

(array([0., 0., 0., 0., 0., 0., 0., 0.]), 0.0)

In [ ]:
scores = cross_val_score(estimator=RidgeClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')
scores, scores.mean()

(array([0., 0., 0., 0., 0., 0., 0., 0.]), 0.0)

In [ ]:
ridge_classifier = RidgeClassifier(random_state=2179)
ridge_classifier.fit(X_train, y_train)
print(classification_report(y_valid, ridge_classifier.predict(X_valid)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      4620
           1       0.00      0.00      0.00       380

    accuracy                           0.92      5000
   macro avg       0.46      0.50      0.48      5000
weighted avg       0.85      0.92      0.89      5000



In [ ]:
roc_auc_score(y_valid, ridge_classifier.predict(X_valid))

0.4998917748917749

## SGDClassifier

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')
scores, scores.mean()

(array([0.47383428, 0.49110399, 0.48300214, 0.50762346, 0.49751068,
        0.47952754, 0.54152184, 0.46434117]), 0.49230813746438745)

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')
scores, scores.mean()

(array([0.02402402, 0.02247191, 0.02076125, 0.14285714, 0.06940063,
        0.0212766 , 0.14074318, 0.00704225]), 0.05607212242944377)

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')
scores, scores.mean()

(array([0.05479452, 0.42857143, 0.10344828, 0.07788595, 0.19298246,
        0.13636364, 0.07695074, 0.04166667]), 0.13908295925113856)

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')
scores, scores.mean()

(array([0.01538462, 0.01153846, 0.01153846, 0.86153846, 0.04230769,
        0.01153846, 0.82307692, 0.00384615]), 0.22259615384615383)

In [ ]:
sgd_classifier = SGDClassifier(random_state=2179)
sgd_classifier.fit(X_train, y_train)
print(classification_report(y_valid, sgd_classifier.predict(X_valid)))

              precision    recall  f1-score   support

          -1       0.92      0.99      0.95      4620
           1       0.07      0.01      0.02       380

    accuracy                           0.91      5000
   macro avg       0.50      0.50      0.49      5000
weighted avg       0.86      0.91      0.88      5000



In [ ]:
roc_auc_score(y_valid, sgd_classifier.predict(X_valid))

0.49954431533378896

### SGDClassifier log loss

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')
scores, scores.mean()

(array([0.47241216, 0.49109568, 0.48705722, 0.51331315, 0.48114672,
        0.49047483, 0.53507004, 0.47027184]), 0.492605205365622)

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')
scores, scores.mean()

(array([0.00675676, 0.01515152, 0.01398601, 0.13636364, 0.02013423,
        0.12959912, 0.13815575, 0.01413428]), 0.05928516231063527)

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')
scores, scores.mean()

(array([0.02777778, 0.5       , 0.07692308, 0.07577055, 0.07894737,
        0.07559257, 0.0754717 , 0.08695652]), 0.12467994497319529)

In [ ]:
scores = cross_val_score(estimator=SGDClassifier(random_state=2179, loss='log'), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')
scores, scores.mean()

(array([0.00384615, 0.00769231, 0.00769231, 0.68076923, 0.01153846,
        0.45384615, 0.81538462, 0.00769231]), 0.24855769230769229)

In [ ]:
sgd_classifier_log = SGDClassifier(random_state=2179, loss='log')
sgd_classifier_log.fit(X_train, y_train)
print(classification_report(y_valid, sgd_classifier_log.predict(X_valid)))

              precision    recall  f1-score   support

          -1       0.92      0.97      0.95      4620
           1       0.05      0.02      0.03       380

    accuracy                           0.90      5000
   macro avg       0.49      0.49      0.49      5000
weighted avg       0.86      0.90      0.88      5000



In [ ]:
roc_auc_score(y_valid, sgd_classifier_log.predict_proba(X_valid)[:, 1])

0.4949419002050581

## RandomForestClassifier

In [ ]:
random_forest_classifier = RandomForestClassifier(oob_score=True, random_state=2179)
random_forest_classifier.fit(X_train, y_train)
random_forest_classifier.oob_score_

0.9257714285714286

In [ ]:
scores = cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')
scores, scores.mean()

(array([0.50492521, 0.49879511, 0.50411681, 0.48576923, 0.49585411,
        0.50802825, 0.50597994, 0.488934  ]), 0.4990503323836657)

In [ ]:
scores = cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')
scores, scores.mean()

(array([0., 0., 0., 0., 0., 0., 0., 0.]), 0.0)

In [ ]:
scores = cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')
scores, scores.mean()

(array([0., 0., 0., 0., 0., 0., 0., 0.]), 0.0)

In [ ]:
scores = cross_val_score(estimator=RandomForestClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')
scores, scores.mean()

(array([0., 0., 0., 0., 0., 0., 0., 0.]), 0.0)

In [ ]:
print(classification_report(y_valid, random_forest_classifier.predict(X_valid)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      4620
           1       0.00      0.00      0.00       380

    accuracy                           0.92      5000
   macro avg       0.46      0.50      0.48      5000
weighted avg       0.85      0.92      0.89      5000



In [ ]:
roc_auc_score(y_valid, random_forest_classifier.predict_proba(X_valid)[:, 1])

0.4870050125313284

## GradientBoostingClassifier

In [ ]:
%%time
scores = cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='roc_auc')
print(scores, scores.mean())

[0.49398089 0.49778668 0.53405686 0.51250119 0.49747151 0.52226377
 0.51896011 0.50801401] 0.510629377374169
CPU times: user 866 ms, sys: 134 ms, total: 1 s
Wall time: 4min 24s


In [ ]:
scores = cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='f1')
scores, scores.mean()

(array([0.00763359, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]), 0.0009541984732824428)

In [103]:
scores = cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='precision')
scores, scores.mean()

(array([0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. ]), 0.0625)

In [104]:
scores = cross_val_score(estimator=GradientBoostingClassifier(random_state=2179), X=X_train, y=y_train, cv=cv, n_jobs=-1, scoring='recall')
scores, scores.mean()

(array([0.00384615, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]), 0.0004807692307692308)

In [ ]:
gb_classifier = GradientBoostingClassifier(random_state=2179)
gb_classifier.fit(X_train, y_train)
print(classification_report(y_valid, gb_classifier.predict(X_valid)))

              precision    recall  f1-score   support

          -1       0.92      1.00      0.96      4620
           1       0.00      0.00      0.00       380

    accuracy                           0.92      5000
   macro avg       0.46      0.50      0.48      5000
weighted avg       0.85      0.92      0.89      5000



In [ ]:
roc_auc_score(y_valid, gb_classifier.predict_proba(X_valid)[:, 1])

0.501481259968102

# Тест

In [ ]:
def write_to_submission_file(predicted_labels, out_file,
                             target='result', index_label="id"):
    # turn predictions into data frame and save as csv file
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(0, predicted_labels.shape[0]),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [ ]:
df_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'orange_small_churn_test_data.csv'), index_col=0)
df_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Var11,Var12,Var13,Var14,Var15,Var16,Var17,Var18,Var19,Var20,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var31,Var32,Var33,Var34,Var35,Var36,Var37,Var38,Var39,Var40,...,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var209,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229,Var230
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,1225.0,7.0,NaN,NaN,NaN,NaN,NaN,2352.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.0,225.0,NaN,0.0,56.0,NaN,NaN,388.08,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6726960.0,NaN,NaN,...,NaN,P1WvyxLp3Z,2Knk1KF,NaN,taul,1K8T,0Xwj,PHNvXy8,xUOdRmdN20,IBm9AvG,NaN,skcI,HLqf,vzJD,NaN,lVqb,NKv3VA1BpP,sBgB,NaN,uKAI,L84s,Ie_5MZs,NaN,xRz9wph,NaN,CWNY2py,jgOV,UYBR,FzaX,UF16siJ,zCkv,APgdzOv,jySVZNlOJy,NaN,ELof,xb3V,6fzt,Zy3gnGM,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,896.0,14.0,NaN,NaN,NaN,NaN,NaN,1560.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0,140.0,NaN,0.0,96.0,NaN,NaN,133.12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,bZkvyxLkBI,RO12,NaN,taul,1K8T,0Xwj,6KF0k8W,V4E_TU9097,NaN,NaN,CZsd,F3hy,vzJD,09_Q,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,NaN,NaN,TDcECyH,PDRj,cJvF,FzaX,ot6oLzk,oslk,IIvC99a,LM8l689qOp,NaN,NaN,xb3V,RAYp,F2FyR07IdsN7I,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,791.0,7.0,NaN,NaN,NaN,NaN,NaN,2824.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,172.0,215.0,NaN,4.0,16.0,NaN,NaN,324.48,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,2988486.0,NaN,NaN,...,NaN,75lTmBtFkL,RO12,SEuy,taul,1K8T,AnrR,ckoNVBU,_jTP8ioIlJ,JnrRQD4,smXZ,PwdO,9_Y1,C6Eu,VpdQ,sYC_,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,Iy8LM_S,NaN,kq0aHkC,laMb,UYBR,FzaX,6VLNqhB,oslk,6YSocsg,LM8l689qOp,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,mj86,NaN
3,NaN,NaN,NaN,NaN,NaN,2296.0,7.0,NaN,NaN,NaN,NaN,NaN,3732.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,480.0,600.0,NaN,10.0,104.0,NaN,NaN,286.96,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6637740.0,NaN,NaN,...,NaN,YddTmBtueT,RO12,SEuy,taul,1K8T,487l,77f44U8,II0S8f9,a1lFLoc,smXZ,SOkz,9_Y1,C6Eu,VpdQ,IYzP,me75fM6ugJ,kIsH,NaN,uKAI,L84s,NhsEn4L,NaN,GccbB3h,NaN,mAjbk_S,qLRt,UYBR,FzaX,tzp8jNM,oslk,5nQ7A2G,jySVZNlOJy,NaN,kG3k,rgKb,RAYp,F2FyR07IdsN7I,am7c,NaN
4,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,mCGq9ayE15,RO12,NaN,taul,1K8T,lK27,esxkA1P,767sa0XN9l,NaN,NaN,kVZt,9_Y1,vm5R,09_Q,NaN,me75fM6ugJ,kIsH,NaN,uKAI,Mtgm,NhsEn4L,NaN,NaN,eGzu,7Wwvby0,XqfQ,UYBR,FzaX,EPImS85,oslk,MI8s5nE,LM8l689qOp,NaN,NaN,7P5s,RAYp,F2FyR07IdsN7I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,NaN,NaN,NaN,NaN,NaN,714.0,7.0,NaN,NaN,NaN,NaN,NaN,2240.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.0,150.0,NaN,10.0,80.0,NaN,NaN,133.12,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,...,NaN,m9SvyxLkBI,RO12,NaN,taul,1K8T,GYbZ,iJzviRg,3X0xOLn,jCrHwxI,NaN,CGbI,9_Y1,lXxA,09_Q,wMei,7M47J5GA0pTYIFxg5uy,kIsH,NaN,uKAI,L84s,4kVnq_T26xq1p,NaN,xXRNJU2,NaN,uZLuDix,vawI,cJvF,Lmli,ch2oGfM,zCkv,P6pu4Vl,LM8l689qOp,NaN,ELof,xb3V,ZI9m,R4y5gQQWY8OodqDV,am7c,NaN
9996,NaN,NaN,NaN,NaN,NaN,812.0,7.0,NaN,NaN,NaN,NaN,NaN,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,136.0,170.0,NaN,0.0,104.0,NaN,NaN,253.52,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4147200.0,NaN,NaN,...,NaN,HYTrjIK12c,2Knk1KF,NaN,taul,1K8T,JLbT,BMnJ89M,5CtupK8,8DWmKv5,NaN,mK1t,9_Y1,7WNq,VpdQ,hAFG,me75fM6ugJ,kIsH,NaN,uKAI,L84s,XfqtO3UdzaXh_,NaN,Oc9VkgP,NaN,XTbPUYD,jQNd,cJvF,FzaX,8d5YWu3,oslk,sXbT3Cb,LM8l689qOp,NaN,ELof,WqMG,RAYp,55YFVY9,mj86,NaN
9997,NaN,NaN,NaN,NaN,NaN,819.0,7.0,NaN,NaN,NaN,NaN,NaN,40.0,NaN,NaN,NaN,NaN,NaN,

In [ ]:
%%time
X_test = preprocessor_pipeline.fit_transform(df_test)

CPU times: user 994 ms, sys: 15.3 ms, total: 1.01 s
Wall time: 1.19 s


In [ ]:
X_test = pd.DataFrame(X_test, columns=preprocessor_pipeline['nan_columns_dropper'].cols).astype('int')
X_test

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var9,Var10,Var11,Var12,Var13,Var14,Var16,Var17,Var18,Var19,Var21,Var22,Var23,Var24,Var25,Var26,Var27,Var28,Var29,Var30,Var33,Var34,Var35,Var36,Var37,Var38,Var40,Var41,Var43,Var44,Var45,Var46,Var47,...,Var189,Var190,Var191,Var192,Var193,Var194,Var195,Var196,Var197,Var198,Var199,Var200,Var201,Var202,Var203,Var204,Var205,Var206,Var207,Var208,Var210,Var211,Var212,Var213,Var214,Var215,Var216,Var217,Var218,Var219,Var220,Var221,Var222,Var223,Var224,Var225,Var226,Var227,Var228,Var229
0,0,0,0,0,0,1225,7,0,0,8,0,2352,0,0,0,0,0,180,225,0,0,56,0,0,388,0,5,0,0,0,0,0,6726960,0,0,0,0,140,0,0,...,264,0,0,131,0,1,16,0,2,825,1672,1394,0,3319,2,93,1,15,9,1,5,0,23,0,4177,0,129,3928,0,6,989,6,302,3,0,0,22,1,18,0
1,0,0,0,0,0,896,14,0,0,8,0,1560,0,0,0,0,0,112,140,0,0,96,0,0,133,0,5,0,0,0,0,0,0,0,0,0,0,140,0,0,...,264,0,0,176,19,1,16,0,2,229,881,4274,0,786,1,93,0,6,10,0,5,0,29,0,451,0,333,2112,1,6,1686,4,577,0,0,0,22,2,8,0
2,0,0,0,0,0,791,7,0,0,8,0,2824,0,0,0,0,0,172,215,0,4,16,0,0,324,0,5,0,0,0,0,0,2988486,0,0,0,0,140,0,0,...,264,0,0,44,19,1,16,0,32,1320,1058,1541,0,1587,0,18,1,17,10,0,5,1,29,0,1357,0,648,4087,0,6,177,4,173,0,0,1,17,2,8,1
3,0,0,0,0,0,2296,7,0,0,8,0,3732,0,0,0,0,0,480,600,0,10,104,0,0,286,0,5,0,0,0,0,0,6637740,0,0,0,0,140,0,0,...,264,0,0,151,19,1,16,0,14,255,535,2582,0,1714,0,18,1,6,10,0,5,0,29,0,1172,0,715,4461,0,6,1833,4,149,3,0,1,17,2,8,0
4,8,0,0,0,0,0,7,28,0,8,0,0,0,0,0,0,0,132,0,0,0,0,0,0,166,0,10,0,0,0,0,0,0,0,14,0,0,140,0,0,...,264,0,0,222,19,1,16,0,142,1390,190,4274,0,2825,0,92,0,6,10,0,5,1,29,0,451,0,86,2824,0,6,432,4,694,0,0,0,4,2,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,714,7,0,0,8,0,2240,0,0,0,0,0,120,150,0,10,80,0,0,133,0,5,0,0,0,0,0,0,0,0,0,0,140,0,0,...,282,0,0,220,19,1,16,0,51,1486,78,3280,0,770,0,81,0,18,4,0,5,0,3,0,4190,0,875,4928,1,8,1316,6,799,0,0,0,22,3,11,0
9996,0,0,0,0,0,812,7,0,0,8,0,2020,0,0,0,0,0,136,170,0,0,104,0,0,253,0,5,0,0,0,0,0,4147200,0,0,0,0,140,0,0,...,264,0,0,100,0,1,16,0,57,402,135,677,0,2948,0,12,1,10,10,0,5,0,38,0,1742,0,380,3907,1,6,272,4,1809,0,0,0,12,2,2,1
9997,0,0,0,0,0,819,7,0,0,8,0,40,0,0,0,0,0,120,150,0,0,56,0,0,321,0,5,0,0,0,0,0,1555200,0,0,0,0,140,0,0,...,264,0,0,45,0,1,16,0,91,1255,1036,829,0,183,0,27,1,14,10,0,5,0,51,0,1353,0,143,3638,1,6,1070,4,6,1,0,0,11,2,2,0
9998,0,0,0,0,0,0,7,0,0,8,0,112,0,0,0,0,0,132,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,140,0,0,...,282,0,0,109,0,1,16,0,88,1523,460,4274,0,690,0,54,1,6,10,0,5,1,38,0,451,0,276,941,0,6,428,4,876,0,0,0,19,2,2,0


In [ ]:
prediction = logistic_regression.predict_proba(X_test)

In [ ]:
write_to_submission_file(prediction[:, 1], 'result.csv')